In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
whole_df = pd.read_csv("/Users/cha/Desktop/Code/eCSpress/data/clean_baseline.csv", sep=";")
whole_df['Date et heure de comptage'] = pd.to_datetime(whole_df['Date et heure de comptage'], format='%Y-%m-%d %H:%M:%S')

print("len whole_df", len(whole_df))
print(whole_df.describe())
# whole_df.head()

len whole_df 27548
       Débit horaire  Taux d'occupation  Etat trafic_Bloqué  \
count   27548.000000       27548.000000        27548.000000   
mean      601.063888          10.564201            0.017460   
std       336.857235          12.082598            0.130982   
min         4.000000           0.000000            0.000000   
25%       339.000000           2.947227            0.000000   
50%       578.000000           6.071945            0.000000   
75%       834.000000          13.340005            0.000000   
max      2431.000000          98.346670            1.000000   

       Etat trafic_Fluide  Etat trafic_Inconnu  Etat trafic_Pré-saturé  \
count        27548.000000         27548.000000            27548.000000   
mean             0.748512             0.046210                0.140664   
std              0.433876             0.209944                0.347680   
min              0.000000             0.000000                0.000000   
25%              0.000000             0.000

catégorie vacances:
- Départ weekend/vacances
- vacances, Été ?
- Jour de la semaine
- saison
- Moment de la journée

### Jours fériés and vacation

In [40]:
def get_date(row):
    return row.date()

feries = pd.read_csv("/Users/cha/Desktop/Code/eCSpress/data/jours_feries_metropole.csv")
feries = feries[feries['annee'].isin([2021, 2022])]
feries['date'] = pd.to_datetime(feries['date']).apply(get_date)
# series_feries = feries['date']
feries = feries.reset_index()
feries

,index,date,annee,zone,nom_jour_ferie
0,209,2021-01-01,2021,Métropole,1er janvier
1,210,2021-04-05,2021,Métropole,Lundi de Pâques
2,211,2021-05-01,2021,Métropole,1er mai
3,212,2021-05-08,2021,Métropole,8 mai
4,213,2021-05-13,2021,Métropole,Ascension
5,214,2021-05-24,2021,Métropole,Lundi de Pentecôte
6,215,2021-07-14,2021,Métropole,14 juillet
7,216,2021-08-15,2021,Métropole,Assomption
8,217,2021-11-01,2021,Métropole,Toussaint
9,218,2021-11-11,2021,Métropole,11 novembre


INCLUT T ON LES PROVINCIAUX ?

In [57]:
vacation = pd.read_csv("/Users/cha/Desktop/Code/eCSpress/data/fr-en-calendrier-scolaire.csv", sep=";")
vacation["start_date"] = pd.to_datetime(vacation["start_date"], format='%Y-%m-%d').apply(get_date)
vacation["end_date"] = pd.to_datetime(vacation["end_date"], format='%Y-%m-%d').apply(get_date)
vacation = vacation[(vacation["zones"]=="Zone C") & (vacation["annee_scolaire"].isin(["2021-2022", "2022-2023"])) & (vacation["location"]=="Paris")]
vacation = vacation.reset_index()
vacation

,index,description,population,start_date,end_date,location,zones,annee_scolaire
0,175,Vacances de la Toussaint,-,2021-10-23,2021-11-08,Paris,Zone C,2021-2022
1,183,Vacances d'Hiver,-,2022-02-19,2022-03-07,Paris,Zone C,2021-2022
2,244,Pont de l'Ascension,-,2023-05-18,2023-05-22,Paris,Zone C,2022-2023
3,449,Vacances de Noël,-,2021-12-18,2022-01-03,Paris,Zone C,2021-2022
4,456,Vacances de Printemps,-,2022-04-23,2022-05-09,Paris,Zone C,2021-2022
5,479,Vacances d'Été,Élèves,2022-07-07,2022-09-01,Paris,Zone C,2021-2022
6,484,Vacances de la Toussaint,-,2022-10-22,2022-11-07,Paris,Zone C,2022-2023
7,499,Vacances de Printemps,-,2023-04-22,2023-05-09,Paris,Zone C,2022-2023
8,767,Vacances de Noël,-,2022-12-17,2023-01-03,Paris,Zone C,2022-2023
9,1077,Début des Vacances d'Été,-,2023-07-08,2023-07-08,Paris,Zone C,2022-2023


### Jour de la semaine

In [20]:
whole_df['Date'] = pd.to_datetime(whole_df["Date et heure de comptage"]).dt.date
whole_df['Jour de la semaine'] = pd.to_datetime(whole_df["Date et heure de comptage"]).dt.dayofweek
whole_df

,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic_Bloqué,Etat trafic_Fluide,Etat trafic_Inconnu,Etat trafic_Pré-saturé,Etat trafic_Saturé,Etat arc_Barré,Etat arc_Invalide,Date,Jour de la semaine
0,AV_Champs_Elysees,2022-01-01 11:00:00,524.0,8.85667,0,1,0,0,0,0,1,2022-01-01,5
1,AV_Champs_Elysees,2022-01-01 07:00:00,491.0,7.71611,0,1,0,0,0,0,1,2022-01-01,5
2,AV_Champs_Elysees,2022-01-01 19:00:00,997.0,46.28889,0,0,0,0,1,0,1,2022-01-01,5
3,AV_Champs_Elysees,2022-01-01 18:00:00,1096.0,46.60222,0,0,0,0,1,0,1,2022-01-01,5
4,AV_Champs_Elysees,2022-01-01 09:00:00,258.0,3.75445,0,1,0,0,0,0,1,2022-01-01,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27543,Sts_Peres,2022-10-31 14:00:00,428.0,6.23000,0,1,0,0,0,0,1,2022-10-31,0
27544,Sts_Peres,2022-08-01 04:00:00,83.0,0.99222,0,1,0,0,0,0,1,2022-08-01,0
27545,Sts_Peres,2022-08-01 03:00:00,143.0,1.72778,0,1,0,0,0,0,1,2022-08-01,0
27546,Sts_Peres,2022-08-01 02:00:00,190.0,2.22778,0,1,0,0,0,0,1,2022-08-01,0


In [21]:
whole_df = pd.concat([
    whole_df,
    pd.get_dummies(whole_df['Jour de la semaine'],
                   prefix='Jour de la semaine',
                   drop_first=False)
], axis=1).drop(columns=['Jour de la semaine'])

whole_df.head()

,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic_Bloqué,Etat trafic_Fluide,Etat trafic_Inconnu,Etat trafic_Pré-saturé,Etat trafic_Saturé,Etat arc_Barré,Etat arc_Invalide,Date,Jour de la semaine_0,Jour de la semaine_1,Jour de la semaine_2,Jour de la semaine_3,Jour de la semaine_4,Jour de la semaine_5,Jour de la semaine_6
0,AV_Champs_Elysees,2022-01-01 11:00:00,524.0,8.85667,0,1,0,0,0,0,1,2022-01-01,0,0,0,0,0,1,0
1,AV_Champs_Elysees,2022-01-01 07:00:00,491.0,7.71611,0,1,0,0,0,0,1,2022-01-01,0,0,0,0,0,1,0
2,AV_Champs_Elysees,2022-01-01 19:00:00,997.0,46.28889,0,0,0,0,1,0,1,2022-01-01,0,0,0,0,0,1,0
3,AV_Champs_Elysees,2022-01-01 18:00:00,1096.0,46.60222,0,0,0,0,1,0,1,2022-01-01,0,0,0,0,0,1,0
4,AV_Champs_Elysees,2022-01-01 09:00:00,258.0,3.75445,0,1,0,0,0,0,1,2022-01-01,0,0,0,0,0,1,0


In [ ]:
whole_df['Jour de la semaine']